In [1]:
import os
import pickle
import glob
import random
import time

import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from sklearn import ensemble, svm
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler, StandardScaler

import npc_lims
from npc_sessions import DynamicRoutingSession
import npc_sessions.utils
from dynamic_routing_analysis import spike_utils, decoding_utils


In [2]:
all_ephys_sessions = tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True, is_annotated=True))

In [3]:
def load_facemap_data(session,session_info,trials,vid_angle,keep_n_SVDs=500,use_s3=True):

    vid_angle_npc_names={
            'behavior':'side',
            'face':'front',
            'eye':'eye',
            }
    
    # path=os.path.join(session_info.allen_path,'processed')

    if use_s3==False:
        if vid_angle=='behavior':
            multi_ROI_path=r"D:\DR Pilot Data\full_video_multi_ROI"
            _dir,vidfilename=os.path.split(glob.glob(os.path.join(session_info.allen_path,"Behavior_*.mp4"))[0])
        elif vid_angle=='face':
            multi_ROI_path=r"D:\DR Pilot Data\full_video_multi_ROI_face"
            _dir,vidfilename=os.path.split(glob.glob(os.path.join(session_info.allen_path,"Face_*.mp4"))[0])

        behav_path = os.path.join(multi_ROI_path,vidfilename[:-4]+'_trimmed_proc.npy')
        behav_info=np.load(behav_path,allow_pickle=True)

        for frame_time in session._video_frame_times:
            if vid_angle_npc_names[vid_angle] in frame_time.name:
                cam_frames=frame_time.timestamps
                break

        facemap_info={}

        #actually keep all ROIs
        #facemap_info['motion']=behav_info.item()['motion']
        facemap_info['motSVD']=behav_info.item()['motSVD']
    #use s3 data
    else:
        # behav_info = np.load(
        #     npc_lims.get_cache_path('facemap',session.id,version='any'),
        #     allow_pickle=True
        # )
        # behav_info = behav_info.item()
        # cam_frames = behav_info['cam_frames']
        camera_to_facemap_name = {
            "face": "Face",
            "behavior": "Behavior",
        }
        motion_svd = npc_sessions.utils.get_facemap_output_from_s3(
                    session.id, camera_to_facemap_name[vid_angle], "motSVD"
                )
        
        for frame_time in session._video_frame_times:
            if vid_angle_npc_names[vid_angle] in frame_time.name:
                cam_frames=frame_time.timestamps
                break

        facemap_info = {
            #'motion': behav_info['motion'],
            'motSVD': motion_svd
        }

    #calculate mean face motion, SVD in 1 sec prior to each trial
    # 1 sec before stimulus onset
    time_before=0.2
    time_after=0
    fps=60

    behav_SVD_by_trial={}
    behav_motion_by_trial={}
    mean_trial_behav_SVD={}
    mean_trial_behav_motion={}

    # trials=pd.read_parquet(
    #             npc_lims.get_cache_path('trials',session.id,version='any')
    #         )

    if use_s3==False:
        for rr in range(0,len(facemap_info['motSVD'])):
            behav_SVD_by_trial[rr] = np.zeros((int((time_before+time_after)*fps),keep_n_SVDs,len(trials)))
            behav_motion_by_trial[rr] = np.zeros((int((time_before+time_after)*fps),len(trials)))

            behav_SVD_by_trial[rr][:]=np.nan
            behav_motion_by_trial[rr][:]=np.nan

            for tt,stimStartTime in enumerate(trials[:]['stim_start_time']):
                if len(np.where(cam_frames>=stimStartTime)[0])>0:
                    stim_start_frame=np.where(cam_frames>=stimStartTime)[0][0]
                    trial_start_frame=int(stim_start_frame-time_before*fps)
                    trial_end_frame=int(stim_start_frame+time_after*fps)
                    if trial_start_frame<facemap_info['motSVD'][rr][:,0].shape[0] and trial_end_frame<facemap_info['motSVD'][rr][:,0].shape[0]:
                        behav_SVD_by_trial[rr][:,:,tt] = facemap_info['motSVD'][rr][trial_start_frame:trial_end_frame,:keep_n_SVDs]    
                        behav_motion_by_trial[rr][:,tt] = facemap_info['motion'][rr][trial_start_frame:trial_end_frame]
                    else:
                        break

            mean_trial_behav_SVD[rr] = np.nanmean(behav_SVD_by_trial[rr],axis=0)
            mean_trial_behav_motion[rr] = np.nanmean(behav_motion_by_trial[rr],axis=0)

    else:
        rr=0
        motsvd=np.asarray(facemap_info['motSVD'][:,:])

        behav_SVD_by_trial[rr] = np.zeros((int((time_before+time_after)*fps),keep_n_SVDs,len(trials)))
        behav_motion_by_trial[rr] = np.zeros((int((time_before+time_after)*fps),len(trials)))

        behav_SVD_by_trial[rr][:]=np.nan
        behav_motion_by_trial[rr][:]=np.nan

        for tt,stimStartTime in enumerate(trials[:]['stim_start_time']):
            if len(np.where(cam_frames>=stimStartTime)[0])>0:
                stim_start_frame=np.where(cam_frames>=stimStartTime)[0][0]
                trial_start_frame=int(stim_start_frame-time_before*fps)
                trial_end_frame=int(stim_start_frame+time_after*fps)
                if trial_start_frame<motsvd[:,0].shape[0] and trial_end_frame<motsvd[:,0].shape[0]:
                    behav_SVD_by_trial[rr][:,:,tt] = motsvd[trial_start_frame:trial_end_frame,:keep_n_SVDs]    
                    # behav_motion_by_trial[rr][:,tt] = facemap_info['motion'][trial_start_frame:trial_end_frame]
                else:
                    break

        mean_trial_behav_SVD[rr] = np.nanmean(behav_SVD_by_trial[rr],axis=0)
        # mean_trial_behav_motion[rr] = np.nanmean(behav_motion_by_trial[rr],axis=0)

    return mean_trial_behav_SVD #mean_trial_behav_motion

In [ ]:
all_ephys_sessions[15]

In [16]:
# use both facemap and units to decode
# fix one and shift the other

savepath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\new_annotations\linear_shift_spikes_and_facemap_face_more_trials_20_svds"

# decoder_results={}
except_dict={}

input_data_type='spikes_and_facemap' #spikes or facemap
vid_angle='face' #behavior, face, eye

exclude_cue_trials=False
central_section='4_blocks_plus'
##TODO: decode the labels from the previous trial - mostly if not using "exclude_cue_trials" ###

n_unit_threshold=20
keep_n_SVDs=20

#set params
spikes_binsize=0.2
spikes_time_before=0.2
spikes_time_after=0.1

decoder_binsize=0.2
decoder_time_before=0.2
decoder_time_after=0.1

start_time=time.time()

#loop through sessions
# session= all_ephys_sessions[10]
for session_info in all_ephys_sessions[:]:
    session=DynamicRoutingSession(session_info.id)
    session_id=str(session_info.id)
    decoder_results={}
    try:
        #load trials and units
        try:
            trials=pd.read_parquet(
                npc_lims.get_cache_path('trials',session_info.id,version='0.0.214')
            )
        except:
            print('no cached trials table, using npc_sessions')
            trials = session.trials[:]

        if exclude_cue_trials:
            trials=trials.query('is_reward_scheduled==False').reset_index()

        if 'spikes' in input_data_type:
            #make data array
            try:
                units=pd.read_parquet(
                    npc_lims.get_cache_path('units',session_info.id,version='0.0.214')
                )
            except:
                print('no cached units table, using npc_sessions')
                units = session.units[:]
            #add probe to structure name
            structure_probe=spike_utils.get_structure_probe(units)
            for uu, unit in units.iterrows():
                units.loc[units['unit_id']==unit['unit_id'],'structure']=structure_probe.loc[structure_probe['unit_id']==unit['unit_id'],'structure_probe']
            
            #make trial data array for baseline activity
            trial_da = spike_utils.make_neuron_time_trials_tensor(units, trials, spikes_time_before, spikes_time_after, spikes_binsize)

        if 'facemap' in input_data_type:
            # mean_trial_behav_SVD,mean_trial_behav_motion = load_facemap_data(session,session_info,trials,vid_angle)
            mean_trial_behav_SVD = load_facemap_data(session,session_info,trials,vid_angle,keep_n_SVDs)
        
        
        #make fake blocks for templeton sessions
        if 'Templeton' in session_info.project:
            start_time=trials['start_time'].iloc[0]
            fake_context=np.full(len(trials), fill_value='nan')
            fake_block_nums=np.full(len(trials), fill_value=np.nan)
            block_context_names=['vis','aud']

            if np.random.choice(block_context_names,1)=='vis':
                block_contexts=['vis','aud','vis','aud','vis','aud']
            else:
                block_contexts=['aud','vis','aud','vis','aud','vis']

            for block in range(0,6):
                block_start_time=start_time+block*10*60
                block_end_time=start_time+(block+1)*10*60
                block_trials=trials[:].query('start_time>=@block_start_time').index
                fake_context[block_trials]=block_contexts[block]
                fake_block_nums[block_trials]=block
            trials['block_index']=fake_block_nums
            trials['context_name']=fake_context

        if central_section=='4_blocks':
            #find middle 4 block labels
            middle_4_block_trials=trials.query('block_index>0 and block_index<5')
            middle_4_blocks=middle_4_block_trials.index.values

            #find the number of trials to shift by, from -1 to +1 block
            negative_shift=middle_4_blocks.min()
            positive_shift=trials.index.max()-middle_4_blocks.max()
            shifts=np.arange(-negative_shift,positive_shift+1)
        elif central_section=='4_blocks_plus':
            #find middle 4 block labels
            first_block=trials.query('block_index==0').index.values
            middle_of_first=first_block[np.round(len(first_block)/2).astype('int')]

            last_block=trials.query('block_index==5').index.values
            middle_of_last=last_block[np.round(len(last_block)/2).astype('int')]

            middle_4_block_trials=trials.loc[middle_of_first:middle_of_last]
            middle_4_blocks=middle_4_block_trials.index.values

            #find the number of trials to shift by, from -1 to +1 block
            negative_shift=middle_4_blocks.min()
            positive_shift=trials.index.max()-middle_4_blocks.max()
            shifts=np.arange(-negative_shift,positive_shift+1)
            # #add 1 block to the end
            # shifts=np.concatenate([shifts,[shifts.max()+1]])

        # #find middle 4 block labels
        # middle_4_block_trials=trials.query('block_index>0 and block_index<5')
        # middle_4_blocks=middle_4_block_trials.index.values

        # #find the number of trials to shift by, from -1 to +1 block
        # negative_shift=middle_4_blocks.min()
        # positive_shift=trials.index.max()-middle_4_blocks.max()
        # shifts=np.arange(-negative_shift,positive_shift+1)

        decoder_results[session_id]={}
        decoder_results[session_id]['shifts'] = shifts
        decoder_results[session_id]['middle_4_blocks'] = middle_4_blocks
        decoder_results[session_id]['spikes_binsize'] = spikes_binsize
        decoder_results[session_id]['spikes_time_before'] = spikes_time_before
        decoder_results[session_id]['spikes_time_after'] = spikes_time_after
        decoder_results[session_id]['decoder_binsize'] = decoder_binsize
        decoder_results[session_id]['decoder_time_before'] = decoder_time_before
        decoder_results[session_id]['decoder_time_after'] = decoder_time_after
        decoder_results[session_id]['input_data_type'] = input_data_type
        if 'facemap' in input_data_type:
            decoder_results[session_id]['vid_angle'] = vid_angle
        decoder_results[session_id]['trials'] = trials
        decoder_results[session_id]['results'] = {}

        ####how to deal with areas for facemap features
        if 'spikes' in input_data_type:
            areas=units['structure'].unique()
            areas=np.concatenate([areas,['all']])
        # elif input_data_type=='facemap':
        #     # areas = list(mean_trial_behav_SVD.keys())
        #     areas=[0]

        decoder_results[session_id]['areas'] = areas

        for aa in areas:
            #make shifted trial data array
            if 'spikes' in input_data_type:
                if aa == 'all':
                    area_units=units
                else:
                    area_units=units.query('structure==@aa')

                n_units=len(area_units)
                if n_units<n_unit_threshold:
                    continue
            
            #keep track of different shifts
            decoder_results[session_id]['results'][aa]={}
            decoder_results[session_id]['results'][aa]['shift_spikes']={}
            decoder_results[session_id]['results'][aa]['shift_facemap']={}

            if 'spikes' in input_data_type:
                
                decoder_results[session_id]['results'][aa]['unit_ids']={}
                decoder_results[session_id]['results'][aa]['n_units']={}
                decoder_results[session_id]['results'][aa]['unit_ids']=area_units['unit_id'].values
                decoder_results[session_id]['results'][aa]['n_units']=len(area_units)

                #find mean ccf location of units
                decoder_results[session_id]['results'][aa]['ccf_ap_mean']=area_units['ccf_ap'].mean()
                decoder_results[session_id]['results'][aa]['ccf_dv_mean']=area_units['ccf_dv'].mean()
                decoder_results[session_id]['results'][aa]['ccf_ml_mean']=area_units['ccf_ml'].mean()


            #loop through shifts
            shift_types=['spikes','facemap']
            for shift_type in shift_types:

                for sh,shift in enumerate(shifts):
                    
                    labels=middle_4_block_trials['context_name'].values

                    if shift_type=='spikes':
                        #shift units
                        shifted_trial_da = trial_da.sel(trials=middle_4_blocks+shift,unit_id=area_units['unit_id'].values).mean(dim='time').values
                        spike_input_data=shifted_trial_da.T

                        #keep facemap aligned
                        trials_used=middle_4_blocks
                        SVD=mean_trial_behav_SVD[0][:,trials_used]
                        facemap_input_data=SVD.T

                        if np.sum(np.isnan(facemap_input_data))>0:
                            incl_inds=~np.isnan(facemap_input_data).any(axis=1)
                            facemap_input_data=facemap_input_data[incl_inds,:]
                            labels=labels[incl_inds]

                    elif shift_type=='facemap':
                        #keep spikes aligned
                        spike_trial_da=trial_da.sel(trials=middle_4_blocks,unit_id=area_units['unit_id'].values).mean(dim='time').values
                        spike_input_data=spike_trial_da.T

                        #shift facemap
                        trials_used=middle_4_blocks+shift
                        shift_exists=[]
                        for tt in trials_used:
                            if tt<mean_trial_behav_SVD[0].shape[1]:
                                shift_exists.append(True)
                            else:
                                shift_exists.append(False)
                        shift_exists=np.array(shift_exists)
                        trials_used=trials_used[shift_exists]

                        SVD=mean_trial_behav_SVD[0][:,trials_used]
                        facemap_input_data=SVD.T

                        if np.sum(np.isnan(facemap_input_data))>0:
                            incl_inds=~np.isnan(facemap_input_data).any(axis=1)
                            facemap_input_data=facemap_input_data[incl_inds,:]
                            labels=labels[incl_inds]

                    #combine input data
                    input_data=np.concatenate([spike_input_data,facemap_input_data],axis=1)

                    decoder_results[session_id]['results'][aa]['shift_'+shift_type][sh]=decoding_utils.linearSVC_decoder(
                            input_data=input_data,
                            labels=labels,
                            crossval='5_fold',
                            crossval_index=None,
                            labels_as_index=True
                        )

            
            print(f'finished {session_id} {aa}')
        #save results
        with open(os.path.join(savepath,session_id+'_decoder_results.pkl'),'wb') as f:
            pickle.dump(decoder_results[session_id],f)

        print(f'finished {session_id}')
        print(f'time elapsed: {time.time()-start_time}')

    except Exception as e:
        print(f'error in {session_id}')
        print(e)
        except_dict[session_id]=e
    


finished 676909_2023-12-14 CA1_probeB
finished 676909_2023-12-14 CA1_probeC
finished 676909_2023-12-14 CA3_probeB
finished 676909_2023-12-14 DG
finished 676909_2023-12-14 MOs_probeD
finished 676909_2023-12-14 MOs_probeA
finished 676909_2023-12-14 MOs_probeE
finished 676909_2023-12-14 ORBl
finished 676909_2023-12-14 ORBm
finished 676909_2023-12-14 ORBvl
finished 676909_2023-12-14 PL
finished 676909_2023-12-14 VISal
finished 676909_2023-12-14 VISp
finished 676909_2023-12-14 all
finished 676909_2023-12-14
time elapsed: 671.0338764190674
finished 676909_2023-12-13 ACAd
finished 676909_2023-12-13 CA1_probeB
finished 676909_2023-12-13 CA3
finished 676909_2023-12-13 DG
finished 676909_2023-12-13 MOp
finished 676909_2023-12-13 MOs_probeD
finished 676909_2023-12-13 MOs_probeA
finished 676909_2023-12-13 ORBvl
finished 676909_2023-12-13 PL
finished 676909_2023-12-13 VISal
finished 676909_2023-12-13 VISp
finished 676909_2023-12-13 all
finished 676909_2023-12-13
time elapsed: 1195.2698829174042
fin

Session 686176_2023-12-06 has known issues: ['https://github.com/AllenInstitute/npc_lims/issues/5']


no cached trials table, using npc_sessions
no cached units table, using npc_sessions


fetching units: 100%|█████████████████████████| 6/6 [04:07<00:00, 41.18s/device]
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  return cls(**config)
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  return cls(**config)
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  return cls(**config)
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  

finished 686176_2023-12-06 CP


...

no cached units table, using npc_sessions


probe_insertions.json and annotation info do not match for 628801_2022-09-20 - using annotation info
fetching units: 100%|█████████████████████████| 3/3 [00:47<00:00, 15.71s/device]
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  return cls(**config)
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  return cls(**config)
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\numcodecs\abc.py:107: UserWarning: Multi-threading is supported for wavpack version>=5.6.4, but current version is 5.5.0. Parallel decoding will not be available.
  return cls(**config)


error in 628801_2022-09-20
No motSVD file found for session 628801_2022-09-20 face camera, despite other paths existing. Check codeocean


C:\Users\ethan.mcbride\AppData\Local\Temp\ipykernel_49024\1684047002.py:116: RuntimeWarning: Mean of empty slice
  mean_trial_behav_SVD[rr] = np.nanmean(behav_SVD_by_trial[rr],axis=0)


error in 628801_2022-09-19
all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 381 and the array at index 1 has size 113
finished 626791_2022-08-17 CA1_probeC
finished 626791_2022-08-17 CA1_probeB
finished 626791_2022-08-17 CA1_probeA
finished 626791_2022-08-17 CP
finished 626791_2022-08-17 DG_probeA
finished 626791_2022-08-17 LP
finished 626791_2022-08-17 MOp
finished 626791_2022-08-17 PoT
finished 626791_2022-08-17 SGN
finished 626791_2022-08-17 TEa
finished 626791_2022-08-17 VISam
finished 626791_2022-08-17 VISl
finished 626791_2022-08-17 VISli
finished 626791_2022-08-17 VISp
finished 626791_2022-08-17 all
finished 626791_2022-08-17
time elapsed: 1715189108.2605271
finished 626791_2022-08-16 CA3
finished 626791_2022-08-16 DG
finished 626791_2022-08-16 MOs
finished 626791_2022-08-16 NB
finished 626791_2022-08-16 ProS
finished 626791_2022-08-16 VISp_probeC
finished 626791_2022-08-16 all
finished 6267

In [17]:
# spike_utils.get_structure_probe(units)

In [18]:
except_dict

{'667252_2023-09-25': IndexError('index 0 is out of bounds for axis 0 with size 0'),
 '666986_2023-08-15': ValueError('all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 455 and the array at index 1 has size 0'),
 '660023_2023-08-09': ValueError('all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 418 and the array at index 1 has size 0'),
 '649944_2023-02-27': ValueError('all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 563 and the array at index 1 has size 562'),
 '644867_2023-02-23': ValueError('all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 447 and the array at index 1 has size 446'),
 '644864_2023-02-02': AssertionError('cluster-ids f

In [ ]:
decoder_results[session_id].keys()

In [ ]:
decoder_results[session_id]['results'].keys()

In [ ]:
shifts=decoder_results[session_id]['shifts']
areas=decoder_results[session_id]['areas']

half_neg_shift=np.round(shifts.min()/2)
half_pos_shift=np.round(shifts.max()/2)
# half_shifts=np.arange(-half_neg_shift,half_pos_shift+1)
half_neg_shift_ind=np.where(shifts==half_neg_shift)[0][0]
half_pos_shift_ind=np.where(shifts==half_pos_shift)[0][0]
half_shift_inds=np.arange(half_neg_shift_ind,half_pos_shift_ind+1) 

bal_acc_spike_shift={}
bal_acc_facemap_shift={}
for aa in areas:
    if aa in decoder_results[session_id]['results']:
        bal_acc_spike_shift[aa]=[]
        bal_acc_facemap_shift[aa]=[]
        for sh in half_shift_inds:
            bal_acc_spike_shift[aa].append(decoder_results[session_id]['results'][aa]['shift_spikes'][sh]['balanced_accuracy'])
            bal_acc_facemap_shift[aa].append(decoder_results[session_id]['results'][aa]['shift_facemap'][sh]['balanced_accuracy'])
        bal_acc_spike_shift[aa]=np.array(bal_acc_spike_shift[aa])
        bal_acc_facemap_shift[aa]=np.array(bal_acc_facemap_shift[aa])


In [ ]:
for aa in areas:
    if aa in decoder_results[session_id]['results']:
        true_acc_spike_shift=bal_acc_spike_shift[aa][shifts[half_shift_inds]==0]
        true_acc_facemap_shift=bal_acc_facemap_shift[aa][shifts[half_shift_inds]==0]
        pval_spike_shift=np.round(np.mean(bal_acc_spike_shift[aa]>=true_acc_spike_shift),decimals=4)
        pval_facemap_shift=np.round(np.mean(bal_acc_facemap_shift[aa]>=true_acc_facemap_shift),decimals=4)
        

        fig,ax=plt.subplots(1,1)
        ax.axhline(true_acc_spike_shift,color='tab:red',linestyle='--',alpha=0.5)
        ax.axhline(true_acc_facemap_shift,color='tab:blue',linestyle='--',alpha=0.5)
        # ax.plot(shifts,bal_acc[aa])
        ax.plot(shifts[half_shift_inds],bal_acc_spike_shift[aa],color='tab:red',label='spike shift')
        ax.plot(shifts[half_shift_inds],bal_acc_facemap_shift[aa],color='tab:blue',label='facemap shift')
        ax.set_xlabel('trial shift')
        ax.set_ylabel('balanced accuracy')
        # ax.set_title(aa+' n='+str(decoder_results[session_id]['results'][aa]['n_units'])+' p='+str(pval))
        ax.set_title(str(aa)+'; p_spike_shift='+str(pval_spike_shift)+'; p_facemap_shift='+str(pval_facemap_shift))
        ax.legend()
        fig.tight_layout()

        